## Is the Swiss franc a safe haven for US investors during recessions?

We focus on the recessions of the last 40 years namely:
1. July 1981- November 1982
2. July 1990 - March 1991
3. March 2001 - November 2001
4. December 2007 - June 2009
5. February 2020 - today. 

Using the S&P 500 returns over these periods and the exchange rate USD/CHF, we run an OLS regression to investigate wether there is a significant correlation.

Load relevant libraries to be used throughout:
- **datetime**: for date handling and manipulation
- **pandas**: to easily work with the data
- **yfinance**: to obtain historical stock price data
- **math**: to use mathematical functions (e.g. ```log(...)```)

In [29]:
from datetime import datetime
import pandas as pd
import yfinance
import math

Through Yahoo! Finance API "wrapper" (yfinance) we create a function to obtain historical data of an asset price at close and return it as a pandas dataframe object.

In [2]:
"""
   Returns historical data of the closing price of ticker between start and end.
   If not passed in, start is set to 01/01/1900 and end the date of today.
   Another optional argument is the frequency which is set to 1 day by default.

   :param string ticker: ticker of the asset which must match the one in yahoo finance
   :param string start: start date of the historical data to be retrived
   :param string end: end date of the historical data to be retrived
   :param string frequency: the epoch between each data point
   :return: historical price at close of ticker between start and end
   :rtype: pandas dataframe
"""
def get_historical_data(ticker, start_date="1900-01-01", end_date=datetime.today().strftime("%Y-%m-%d"), frequency="1d"):
    if start_date > end_date:
        raise Exception("start date cannot be before end date")
    asset = yfinance.Ticker(ticker)
    stock_history = asset.history(period="max", interval=frequency)
    stock_history['Date'] = stock_history.index.strftime("%Y-%m-%d")
    stock_history.reset_index(drop=True, inplace=True)
    filtered_data = stock_history.loc[(stock_history["Date"] >= start_date) & (
        stock_history["Date"] <= end_date)][["Date", "Close"]]

    return filtered_data

In theory, we could make use of the function ```get_historical_data(...)``` to obtain historical rates for the pair USD/CHF. However, the earliest data point starts in 2003 which would not be enough to analyze the first three recessions. Therefore, we load an excel file to a pandas dataframe with the historical rates of the pair considered. The data was retrieved from Bloomberg.

In [13]:
"""
   Returns historical data of the closing price of the pair USD/CHF between start and end.
   If not passed in, start is set to 01/01/1900 and end the date of today.

   :param string start: start date of the historical data to be retrived
   :param string end: end date of the historical data to be retrived
   :return: historical price at close of ticker between start and end
   :rtype: pandas dataframe
"""
def load_USDCHF_historical_data(start_date="1900-01-01", end_date=datetime.today().strftime("%Y-%m-%d")):
    if start_date > end_date:
        raise Exception("start date cannot be before end date")
    pair_data = pd.read_excel("USDCHF.xlsx").sort_values(by="Date", ascending=True)
    pair_data["Date"] = pair_data["Date"].map(lambda x: x.strftime("%Y-%m-%d"))
    return pair_data.loc[(pair_data["Date"] >= start_date) & (
        pair_data["Date"] <= end_date)][["Date", "Close"]]


Calculate log-returns of asset and define a mapping to be used in the dataframe manipulation.

In [37]:
def log_returns(stock_prices):
    mapping = {stock_prices[0] : 0}
    for i in range(1, len(stock_prices)):
        mapping[stock_prices[i]] = math.log(stock_prices[i] / stock_prices[i-1])
    return mapping

Obtain all the relevant data for all the 5 recessions and ensure dates match (CHF/USD had more data points than SPX which needed to be removed).

In [63]:
recession_dates = [["1981-07-01", "1982-11-01"],
                   ["1990-07-01", "1991-03-01"],
                   ["2001-03-01", "2001-11-01"],
                   ["2007-12-01", "2009-06-01"],
                   ["2020-02-01", "2022-11-21"]]

spx_yf_ticker = "^GSPC"
spx_historical_data = {}
fx_pair_historical_data = {}
n_recession = 0

for recession_date in recession_dates:
    n_recession += 1
    start_date = recession_date[0]
    end_date = recession_date[1]
    
    spx_data = get_historical_data(spx_yf_ticker, start_date, end_date)
    pair_data = load_USDCHF_historical_data(start_date, end_date)

    # replace stock prices with log returns 
    spx_data["Close"] = spx_data["Close"].map(
        log_returns(list(spx_data["Close"])))
    spx_data.index = range(len(spx_data.index))

    
    # clean fx pair data
    pair_data = pair_data[pair_data["Date"].isin(spx_data["Date"])]
    # similarly for pair rates, replace with log returns
    pair_data["Close"] = pair_data["Close"].map(
        log_returns(list(pair_data["Close"])))
    pair_data.index = range(len(pair_data.index))

    spx_historical_data[f"recession {n_recession}"] = spx_data
    fx_pair_historical_data[f"recession {n_recession}"] = pair_data

In [64]:
print(spx_historical_data["recession 1"])
print(fx_pair_historical_data["recession 1"])


           Date     Close
0    1981-07-01  0.000000
1    1981-07-02 -0.008746
2    1981-07-06 -0.009922
3    1981-07-07  0.006807
4    1981-07-08  0.000624
..          ...       ...
334  1982-10-26  0.008663
335  1982-10-27  0.006005
336  1982-10-28 -0.012645
337  1982-10-29  0.000973
338  1982-11-01  0.013002

[339 rows x 2 columns]
           Date     Close
0    1981-07-01  0.000000
1    1981-07-02 -0.000723
2    1981-07-06  0.008404
3    1981-07-07  0.000239
4    1981-07-08  0.002367
..          ...       ...
334  1982-10-26 -0.001366
335  1982-10-27 -0.001368
336  1982-10-28  0.009083
337  1982-10-29 -0.002036
338  1982-11-01 -0.007503

[339 rows x 2 columns]
